### Código para acesso e pré-processamento dos dados do Hidroweb (ANA)

#### Criado por:
* Cristiano Almeida
* Filipe Lemos
* Lindemberg Barbosa

#### Em: 11-Out-2022

    Script para acesso ao dados do Hidroweb (da ANA - Agência Nacional de Águas) e padronização da estrutura dos dados.
    
    
**Objetivos:** 
* Baixar todos os dados do Hidroweb
* Organizar os dados em modelo padrão
* Criar os arquivos hdf com esses dados brutos
* Fazer um diagnóstico mínimo e talvez a partir daí um filtro mínimo
* Quantidade de estações por estado
* Quantidade de estações por agência ou órgão de monitoramento
* Quantidade de estações por estado
* Quantidade de estações por ano
* xx O que mais???


#### Dependências:
* Hydrobr - Pacote desenvolvido por Wallisson Moreira de Carvalho (em 2021) e publicado no seu github (https://wallissoncarvalho.github.io/HydroBr/)
* Pandas
* Geopandas
----

### Estrutura dos dados

   Os dados baixados do Hidroweb por meio HydroBr vêm na forma de uma lista de **estações pluviométricas** com as seguintes colunas:
* Name - Nome da estação de monitoramento
* Code - Código da estação de monitoramento
* Type - Tipo de estações 1 (Estações fluviométricas) & 2 (Estações pluviométricas)
* SubBasin - Código da sub-bacia hidrográfica (xx ver na ANA quais são esses códigos)
* City - Cidade onde a estação está localizada
* State - Estado da Federação do Brasil, mas devemos observar que outros países estão incluídos aí (Argentina, Peru, Bolívia, Paraguai, Guiana Francesa & Suriname
* Responsible - Instituição responsável pela operação da estação (ANA, INMET, etc. etc., xx)
* Latitude - Latitude em graus decimais
* Longitude - Longitude em graus decimais
* StartDate - Data da 1ª medição
* EndDate - Data da última medição
* NYD - Quantidade de anos com dados, renomeado para Years
* MD - Porcentagem de dados faltosos entre a data inicial e final (xx tem que ver se é de dados, dias, meses, etc.) missing data percentage between the start and the last date (MD), renomeado para Percentage_of_missing_data
* N_YWOMD - Número de anos sem qualquer falha, renomeado para Years_without_gaps
* YWMD - Porcentagem de anos com dados faltosos, renomeado para Percentage_of_missing_years

    A série temporal, que é um pandas.DataFrame, tem o seguinte formato:
* o index do dataframe é a data
* Os dados de precipitação diárias são do tipo decimal
* Cada coluna corresponde aos dados diários de precipitação de um posto pluviométrico da lista usada na requisição
* A primeira linha do dataframe é o código de cada posto pluviométrico

In [18]:
# pip install hydrobr

In [19]:
%config Completer.use_jedi = False

In [1]:
import hydrobr
import numpy as np
import geopandas
import pandas as pd
import matplotlib.pyplot as plt
import glob
import os
from datetime import datetime

In [2]:
def save_files(df_year_data, df_year_info, filename):
    df_year_data.to_hdf(general_path + '/1 - Organized data/'+ filename, 'table_data', append=True,
                        complevel=9,encoding="cp860")
    df_year_info.to_hdf(general_path + '/1 - Organized data/'+ filename, 'table_info', append=True,
                        complevel=9,encoding="cp860")

#### Variáveis utilizadas

In [3]:
# Caminhos dos diretórios raiz
# general_path = 'C:/Users/cnalm/OneDrive/Hidroweb'    # Caminho do projeto (Cristiano)
general_path = 'D:/Dados_Nuvem/OneDrive/Hidroweb'   # Colocar aqui o caminho de Filipe
# general_path = 'C:/Users/linde/OneDrive/Hidroweb'   # Colocar aqui o caminho de Lindemberg

In [6]:
# List of brazilian states
states = ['PARÁ', 'AMAPÁ', 'RORAIMA', 'AMAZONAS', 'MARANHÃO', 'CEARÁ', 'PIAUÍ', 'PERNAMBUCO', 
          'RIO GRANDE DO NORTE', 'TOCANTINS', 'PARAÍBA', 'ACRE', 'ALAGOAS', 'BAHIA', 'MATO GROSSO',
          'RONDÔNIA', 'SERGIPE', 'GOIÁS', 'MINAS GERAIS', 'DISTRITO FEDERAL', 'MATO GROSSO DO SUL',
          'ESPÍRITO SANTO', 'SÃO PAULO', 'RIO DE JANEIRO', 'PARANÁ', 'SANTA CATARINA', 'RIO GRANDE DO SUL']

states = ['SANTA CATARINA', 'RIO GRANDE DO SUL']

# Countries from South America with data in the Hidroweb (to be deleted)
countries = ['BOLÍVIA', 'PERU', 'PARAGUAI', 'ARGENTINA', 'GUIANA FRANCESA', 'SURINAME']

# Agencies responsible for rainfall monitoring (only in Brazil)
agencies = ['INMET', 'ANA', 'IDESP', 'ICOMI', 'SUDENE', 'EMBRAPA/IPAGRO', 'DEPV', 'CHM', 'OUTRAS', 
            'FUNCEME-CE', 'DNOCS', 'SEMARH-PI', 'CELPA', 'SUDAM', 'EMATER', 'DNOS', 'ELETRONORTE', 
            'EMPARN', 'CPRM', 'COGERH-CE', 'AESA-PB', 'CEMADEN', 'RFFSA', 'IAA', 'APAC-PE', 'SANTO ANTÔNIO',
            'TRAPICHE', 'BRENNAND', 'SERRA GRANDE', 'CODEVASF', 'SEDAM-RO', 'SEMARH-AL', 'NOVO MUNDO',
            'FASE', 'INEMA-BA', 'SABA', 'EGP', 'CEPLAC', 'FABA', 'EMBASA-BA', 'USIBA', 'CIA', 'CERB', 
            'ENERPEIXE', 'FURNAS', 'ICB', 'IGUAÇU', 'CHESF', 'CONS.MANSO', 'CEMIG', 'CAESB', 'COSTA BARROS',
            'ADASA', 'CEMAT', 'CONS.JAURU', 'DNIT', 'VALE', 'COPASA-MG', 'CONS.CORUMBÁIII', 'FAZ.GIRASSOL',
            'CORUMBÁ COM', 'GIOVENARDI', 'DAEE-MG', 'GOIAS SUL', 'CELG', 'STAHELENAYPE', 'KINROSS', 'IEMA-ES',
            'SIGMA', 'TRÊS MARIAS', 'CBA', 'ENERGEST', 'SALTO GRANDE', 'LIGHT', 'DAEE-SP', 'INDAIAZINHO', 
            'INDAIÁ GRANDE', 'STATKRAFT', 'ALUMINAS', 'MORRO VELHO', 'ANGLO GOLD', 'EMAE', 'VOTORCIM II', 
            'CHIMAY', 'RIO PARANÁ', 'DCES', 'QUANTA', 'ZÉ TUNIN', 'BRAÚNA', 'CBCC', 'L.D.Q.S.P.E.', 
            'CEMIG CAMARGOS', 'GERAL', 'AES TIETÊ', 'PRADA', 'ARATU', 'CESP', 'INEA-RJ', 'CEDAE', 'CFLCL',
            'INFOPER', 'ÁGUAS IMPERADOR', 'COPPE/UFRJ', 'CERV-PET', 'M. SERRAT', 'SF ENERGÉTICA', 'BONFANTE',
            'SECRETÁRIO', 'COHIDRO', 'IGAM-MG', 'SPE CRISTINA', 'CPFL-FERROLIGAS', 'MOHINI', 'JAYADITYA', 
            'FLP', 'SANTA CRUZ', 'PARANAPANEMA', 'AGUASPARANÁ', 'CONS.CANOAS', 'COPEL', 'ITAIPUSABESP', 
            'PIRAPORA', 'IAPAR', 'CMNP', 'CONS.CECS', 'KLABIN', 'TIBAGI', 'MOURÃO', 'RIO BRANCO', 'MANASA',
            'ELEJOR', 'ENGIE', 'FOZ DO CHOPIM', 'EPAGRI-SC', 'CELESC', 'CONS.CESAP', 'SENS', 'CONS.ITÁ', 
            'ENERCAN', 'PCH AGUTI', 'ELETROSUL', 'CEEE', 'CONS.CAPIVARI', 'DEPRC', 'BAESA', 'HIDROTÉRMICA',
            'CERAN', 'CAROLINA', 'ELETROCAR', 'DEMEI', 'COOPERLUZ', 'IRGA', 'SOPRS', 'SEMA-RS', 'JAGUARI', 'CLM']

dict_agency = ({'agency':['lixo'],
                'amount of gauges':[-99]
               })
df_gauges_per_agencies = pd.DataFrame(dict_agency)

dict_state = ({'state':['lixo'],
               'amount of gauges':[-99]
               })
df_gauges_per_state = pd.DataFrame(dict_state)

## Revisar as variáveis daqui para baixo

# years=['2014','2015','2016','2017','2018','2019', '2020','2021']
years=['2024']
# df_year = pd.DataFrame(columns=['COD', 'CIDADE', 'ESTADO', 'BAIRRO', 'LAT', 'LONG','DATA','HORA','PREC'])
# df_null = pd.DataFrame({'COD': 'XXXXXXXXX', 'CIDADE':'NULL','ESTADO':'BR','BAIRRO':'NULL','LAT':[-99.99],
#                         'LONG':[-99.99],'DATA':'2000/01/01','HORA':'00:00:00','PREC':[-99.99]})

df_year = pd.DataFrame(columns=['municipio', 'codEstacao', 'uf', 'nomeEstacao', 'latitude', 'longitude',
                                'datahora','valorMedida'])
df_null = pd.DataFrame({'municipio':'NULL', 'codEstacao': 'XXXXXXXXX', 'uf':'BR','nomeEstacao':'NULL',
                        'latitude':[-99.99], 'longitude':[-99.99],'datahora':'2000/01/01 00:00:00','valorMedida':[-99.99]})



In [7]:
df_all_gauges = hydrobr.get_data.ANA.list_prec_stations() 
df_all_gauges

,Name,Code,Type,SubBasin,City,State,Responsible,Latitude,Longitude,StartDate,EndDate,NYD,MD,N_YWOMD,YWMD
0,SALINÓPOLIS,00047000,2,32,SALINÓPOLIS,PARÁ,INMET,-0.6500,-47.5500,1958/01/01,1964/12/31,7,25.0,0,100.0
1,SALINÓPOLIS,00047002,2,32,SALINÓPOLIS,PARÁ,ANA,-0.6231,-47.3536,1977/12/09,2019/08/31,43,3.5,35,18.6
2,CURUÇA,00047003,2,32,CURUÇA,PARÁ,ANA,-0.7375,-47.8536,1981/07/01,2019/07/31,39,2.4,29,25.6
3,PRIMAVERA,00047004,2,32,PRIMAVERA,PARÁ,ANA,-0.9294,-47.0994,1982/02/18,2019/08/31,38,0.0,35,7.9
4,MARUDA,00047005,2,32,MARAPANIM,PARÁ,ANA,-0.6336,-47.6583,1989/08/21,2019/07/31,31,5.0,20,35.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11971,NOVA ESPERANÇA/MARCO BV-8,08461000,2,14,PACARAIMA,RORAIMA,ANA,4.4883,-61.1297,1984/03/23,2019/06/30,36,12.3,13,63.9
11972,MISSÃO AUARIS - JUSANTE,08464001,2,14,BOA VISTA,RORAIMA,ANA,4.0031,-64.4431,1995/04/01,2019/06/30,25,4.2,15,40.0
11973,WILLIAM KRAANPLEIN,08555060,2,90,SURINAME,SURINAME,SURINAME,5.8000,-55.1667,1935/08/01,1948/12/31,14,9.2,7,50.0
11974,ZANDERIJ,08555096,2,90,SURINAME,SURINAME,SURINAME,5.4700,-55.2000,2009/01/31,2010/01/31,2,96.7,0,100.0


In [8]:
# Deleting countries from the rain gauge list
filt = ~df_all_gauges['State'].isin(countries)
df_gauges = df_all_gauges[filt]
df_gauges

,Name,Code,Type,SubBasin,City,State,Responsible,Latitude,Longitude,StartDate,EndDate,NYD,MD,N_YWOMD,YWMD
0,SALINÓPOLIS,00047000,2,32,SALINÓPOLIS,PARÁ,INMET,-0.6500,-47.5500,1958/01/01,1964/12/31,7,25.0,0,100.0
1,SALINÓPOLIS,00047002,2,32,SALINÓPOLIS,PARÁ,ANA,-0.6231,-47.3536,1977/12/09,2019/08/31,43,3.5,35,18.6
2,CURUÇA,00047003,2,32,CURUÇA,PARÁ,ANA,-0.7375,-47.8536,1981/07/01,2019/07/31,39,2.4,29,25.6
3,PRIMAVERA,00047004,2,32,PRIMAVERA,PARÁ,ANA,-0.9294,-47.0994,1982/02/18,2019/08/31,38,0.0,35,7.9
4,MARUDA,00047005,2,32,MARAPANIM,PARÁ,ANA,-0.6336,-47.6583,1989/08/21,2019/07/31,31,5.0,20,35.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11969,ÁGUA FRIA,08460003,2,14,UIRAMUTA,RORAIMA,ANA,4.6428,-60.4964,1996/03/02,2018/04/30,23,1.8,16,30.4
11970,UIRAMUTA,08460004,2,14,UIRAMUTA,RORAIMA,ANA,4.5986,-60.1664,2002/07/13,2019/07/31,18,2.3,12,33.3
11971,NOVA ESPERANÇA/MARCO BV-8,08461000,2,14,PACARAIMA,RORAIMA,ANA,4.4883,-61.1297,1984/03/23,2019/06/30,36,12.3,13,63.9
11972,MISSÃO AUARIS - JUSANTE,08464001,2,14,BOA VISTA,RORAIMA,ANA,4.0031,-64.4431,1995/04/01,2019/06/30,25,4.2,15,40.0


In [9]:
type(df_gauges)

pandas.core.frame.DataFrame

In [10]:
# Renaming some columns
df_gauges.rename(columns = {'NYD':'Years', 'MD':'Percentage_of_missing_data', 'N_YWOMD':'Years_without_gaps', 
                     'YWMD':'Percentage_of_missing_years'}, inplace = True)
df_gauges

,Name,Code,Type,SubBasin,City,State,Responsible,Latitude,Longitude,StartDate,EndDate,Years,Percentage_of_missing_data,Years_without_gaps,Percentage_of_missing_years
0,SALINÓPOLIS,00047000,2,32,SALINÓPOLIS,PARÁ,INMET,-0.6500,-47.5500,1958/01/01,1964/12/31,7,25.0,0,100.0
1,SALINÓPOLIS,00047002,2,32,SALINÓPOLIS,PARÁ,ANA,-0.6231,-47.3536,1977/12/09,2019/08/31,43,3.5,35,18.6
2,CURUÇA,00047003,2,32,CURUÇA,PARÁ,ANA,-0.7375,-47.8536,1981/07/01,2019/07/31,39,2.4,29,25.6
3,PRIMAVERA,00047004,2,32,PRIMAVERA,PARÁ,ANA,-0.9294,-47.0994,1982/02/18,2019/08/31,38,0.0,35,7.9
4,MARUDA,00047005,2,32,MARAPANIM,PARÁ,ANA,-0.6336,-47.6583,1989/08/21,2019/07/31,31,5.0,20,35.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11969,ÁGUA FRIA,08460003,2,14,UIRAMUTA,RORAIMA,ANA,4.6428,-60.4964,1996/03/02,2018/04/30,23,1.8,16,30.4
11970,UIRAMUTA,08460004,2,14,UIRAMUTA,RORAIMA,ANA,4.5986,-60.1664,2002/07/13,2019/07/31,18,2.3,12,33.3
11971,NOVA ESPERANÇA/MARCO BV-8,08461000,2,14,PACARAIMA,RORAIMA,ANA,4.4883,-61.1297,1984/03/23,2019/06/30,36,12.3,13,63.9
11972,MISSÃO AUARIS - JUSANTE,08464001,2,14,BOA VISTA,RORAIMA,ANA,4.0031,-64.4431,1995/04/01,2019/06/30,25,4.2,15,40.0


In [11]:
# Adjusting collumn types
df_gauges['StartDate'] = pd.to_datetime(df_gauges['StartDate'])
df_gauges['EndDate'] = pd.to_datetime(df_gauges['EndDate'])

In [12]:
df_gauges.describe()
df_gauges.dtypes
df_gauges

,Name,Code,Type,SubBasin,City,State,Responsible,Latitude,Longitude,StartDate,EndDate,Years,Percentage_of_missing_data,Years_without_gaps,Percentage_of_missing_years
0,SALINÓPOLIS,00047000,2,32,SALINÓPOLIS,PARÁ,INMET,-0.6500,-47.5500,1958-01-01,1964-12-31,7,25.0,0,100.0
1,SALINÓPOLIS,00047002,2,32,SALINÓPOLIS,PARÁ,ANA,-0.6231,-47.3536,1977-12-09,2019-08-31,43,3.5,35,18.6
2,CURUÇA,00047003,2,32,CURUÇA,PARÁ,ANA,-0.7375,-47.8536,1981-07-01,2019-07-31,39,2.4,29,25.6
3,PRIMAVERA,00047004,2,32,PRIMAVERA,PARÁ,ANA,-0.9294,-47.0994,1982-02-18,2019-08-31,38,0.0,35,7.9
4,MARUDA,00047005,2,32,MARAPANIM,PARÁ,ANA,-0.6336,-47.6583,1989-08-21,2019-07-31,31,5.0,20,35.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11969,ÁGUA FRIA,08460003,2,14,UIRAMUTA,RORAIMA,ANA,4.6428,-60.4964,1996-03-02,2018-04-30,23,1.8,16,30.4
11970,UIRAMUTA,08460004,2,14,UIRAMUTA,RORAIMA,ANA,4.5986,-60.1664,2002-07-13,2019-07-31,18,2.3,12,33.3
11971,NOVA ESPERANÇA/MARCO BV-8,08461000,2,14,PACARAIMA,RORAIMA,ANA,4.4883,-61.1297,1984-03-23,2019-06-30,36,12.3,13,63.9
11972,MISSÃO AUARIS - JUSANTE,08464001,2,14,BOA VISTA,RORAIMA,ANA,4.0031,-64.4431,1995-04-01,2019-06-30,25,4.2,15,40.0


In [13]:
data_state = pd.pivot_table(df_gauges[['Code', 'State']], values='Code', index=['State'], aggfunc=np.count_nonzero)
data_state.sort_values('Code', inplace=True, ascending=False)
data_state.reset_index(inplace=True)
data_state

,State,Code
0,SÃO PAULO,2271
1,BAHIA,1226
2,PARANÁ,1185
3,MINAS GERAIS,1150
4,CEARÁ,1149
5,RIO DE JANEIRO,595
6,RIO GRANDE DO SUL,540
7,PERNAMBUCO,374
8,PARAÍBA,360
9,PIAUÍ,321


### Downloading rainfall daily data

In [14]:
# df_temp = df_gauges.copy(deep = True)
# df_temp
state = 'RIO GRANDE DO SUL'
# filt = ((df_gauges['State']==state) & (df_gauges['Responsible']=='CEMADEN'))
filt = ((df_gauges['State']==state))
df_temp = df_gauges[filt]
df_temp

# filt = (df_gauges['Responsible']=='DAEE-SP')
# gauges_aesa = df_gauges[filt]
# gauges_aesa.head()

,Name,Code,Type,SubBasin,City,State,Responsible,Latitude,Longitude,StartDate,EndDate,Years,Percentage_of_missing_data,Years_without_gaps,Percentage_of_missing_years
11259,MARCELINO RAMOS,02751005,2,73,MARCELINO RAMOS,RIO GRANDE DO SUL,ELETROSUL,-27.4611,-51.9136,1935-09-01,1992-10-31,58,37.9,23,60.3
11260,PAIM FILHO,02751006,2,72,PAIM FILHO,RIO GRANDE DO SUL,ANA,-27.7125,-51.7378,1957-10-17,2019-10-31,63,1.4,57,9.5
11261,SANANDUVA,02751007,2,72,SANANDUVA,RIO GRANDE DO SUL,ANA,-27.9828,-51.7836,1957-07-03,2019-10-31,63,1.5,55,12.7
11262,PCH FORQUILHA BARRAMENTO,02751008,2,72,MAXIMILIANO DE ALME,RIO GRANDE DO SUL,CEEE,-27.6589,-51.7483,1949-06-01,1981-12-31,33,0.0,32,3.0
11267,BARRACÃO,02751015,2,70,BARRACÃO,RIO GRANDE DO SUL,ANA,-27.6786,-51.4547,1977-12-01,2019-10-31,43,1.8,27,37.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11854,CORDÃO,03353002,2,88,SANTA VITÓRIA DO PA,RIO GRANDE DO SUL,ANA,-33.1000,-53.0500,1967-05-01,1975-04-30,9,4.2,3,66.7
11855,SANTA VITÓRIA - PAU FINCADO,03353004,2,88,SANTA VITÓRIA DO PA,RIO GRANDE DO SUL,CLM,-33.1167,-53.2000,1967-03-01,1976-04-30,10,6.7,5,50.0
11856,SANTA AMÉLIA,03353006,2,88,SANTA VITÓRIA DO PA,RIO GRANDE DO SUL,CLM,-33.3667,-53.1833,1971-01-01,1981-05-31,11,1.6,6,45.5
11857,SANTA VITÓRIA DO PALMAR,03353007,2,88,SANTA VITÓRIA DO PA,RIO GRANDE DO SUL,INMET,-33.5325,-53.3494,1934-01-01,1998-12-31,65,16.0,32,50.8


In [36]:
gauge_codes = df_temp['Code'].values.tolist()

In [37]:
gauge_codes_len = str(len(gauge_codes))

count = 0
error = 0
code_list = []

for gauge_code in gauge_codes:
    try:
        rainfall_daily_data = hydrobr.get_data.ANA.prec_data([gauge_code])
        # rainfall_daily_data.to_hdf(general_path + '/1 - Organized data/'+ gauge_code + '.h5', 'table_data', append = False, complevel = 9, encoding="cp860")
    except:
        print('gauge error')
        error = error + 1
        code_list.append(gauge_code)
    print(gauge_code)
    count = count + 1
    print(str(count)+"/"+gauge_codes_len)
print(str(error) + " download errors")
print('stations with download error\n', code_list)
print(len(code_list))

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.07s/it]


00634000
1/360


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.31s/it]


00634001
2/360


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.74s/it]


...

00836001
358/360


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:04<00:00,  4.01s/it]


00837000
359/360


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.52s/it]

00837001
360/360
22 download errors
stations with download error
 ['00635134', '00635135', '00637074', '00637075', '00734044', '00734045', '00734046', '00735186', '00735197', '00735198', '00735199', '00735200', '00735207', '00735208', '00735209', '00735210', '00735211', '00735212', '00735213', '00735214', '00735215', '00735216']
22


In [34]:
# if (len(code_list) > 0):
#     df_error = pd.DataFrame(code_list)
#     df_error.columns = ['Code']
#     df_error.to_hdf(general_path + '/Download Error/'+ state + '_download_error.h5', 'table_info', append = False, complevel = 9, encoding="cp860")
#     print(df_error)
# else:
#     print("No errors")